In [2]:
import numpy as np
import pandas as pd
import os

<h1>Daten Einlesen</h1>

In [3]:
unpickled_df = pd.read_pickle("../Data/data_inside_20211012_000102.pkl")

print(unpickled_df)

{'accelerometer_x': array([-7.2265625, -7.1679688, -7.1679688, ..., -7.084961 , -7.084961 ,
       -7.1679688], dtype=float32), 'accelerometer_y': array([6.850586, 6.801758, 6.801758, ..., 6.821289, 6.821289, 6.928711],
      dtype=float32), 'accelerometer_z': array([0.9082031 , 0.7861328 , 0.7861328 , ..., 0.8496094 , 0.8496094 ,
       0.88378906], dtype=float32), 'magnetometer_x': array([-53937500., -53937500., -53937500., ..., -51750000., -51750000.,
       -54312500.], dtype=float32), 'magnetometer_y': array([4062500., 4062500., 4062500., ..., 3750000., 3750000., 3750000.],
      dtype=float32), 'magnetometer_z': array([-5937500., -5937500., -5937500., ..., -6000000., -6000000.,
       -4812500.], dtype=float32), 'audio': array([0.00872803, 0.0083313 , 0.00796509, ..., 0.0163269 , 0.0163269 ,
       0.01504517], dtype=float32), 'timer': array([       0,        1,        1, ..., 86277649, 86277650, 86277650],
      dtype=int32)}


In [3]:
print(unpickled_df['timer'].max())

86277650


<h1>Audio Step für Smoothing festlegen</h1>

In [4]:
# to 20Hz
# Calculation:   audio_step  =  2100d/s  /  20Hz  =  105s

audio_step = int(2100 / 20) 

print('audio steps:', audio_step)

audio steps: 105


<h1>Indizes für den Split der Sensor Dateien finden</h1>

In [5]:
dict_no_audio = {}
for k in unpickled_df.keys():
    if k != 'audio':
        dict_no_audio[k] = unpickled_df[k]

print(dict_no_audio['accelerometer_x'].size, ' / ', dict_no_audio['magnetometer_x'].size, ' / ', dict_no_audio['timer'].size)
############################################
# das hier ist ist hoffentlich nur für das 1h example notwendig
#dict_no_audio['timer'] = np.append(dict_no_audio['timer'], 3599906)
#print('after append: ', dict_no_audio['accelerometer_x'].size, ' / ', dict_no_audio['magnetometer_x'].size, ' / ', dict_no_audio['timer'].size)
############################################

df_no_audio = pd.DataFrame(dict_no_audio)
df_no_audio.set_index('timer', inplace=True)
df_no_audio['count'] = np.arange(len(df_no_audio))
df_no_audio

177331479  /  177331479  /  177331479


,accelerometer_x,accelerometer_y,accelerometer_z,magnetometer_x,magnetometer_y,magnetometer_z,count
timer,,,,,,,
0,-7.226562,6.850586,0.908203,-53937500.0,4062500.0,-5937500.0,0
1,-7.167969,6.801758,0.786133,-53937500.0,4062500.0,-5937500.0,1
1,-7.167969,6.801758,0.786133,-53937500.0,4062500.0,-5937500.0,2
2,-7.167969,6.801758,0.786133,-53937500.0,4062500.0,-5937500.0,3
2,-7.177734,6.752930,0.834961,-53937500.0,4062500.0,-5937500.0,4
...,...,...,...,...,...,...,...
86277647,-7.202148,7.001953,0.859375,-51750000.0,3750000.0,-6000000.0,177331474
86277648,-7.036133,6.767578,0.839844,-51750000.0,3750000.0,-6000000.0,177331475
86277649,-7.084961,6.821289,0.849609,-51750000.0,3750000.0,-6000000.0,177331476


In [7]:
# range von timer values an denen aufgeteilt werden muss
# enthält immer die letzte ms die in einem Step sein soll
val_range = np.array(range(50,unpickled_df['timer'].max(),50))
print (val_range, val_range.size)

[      50      100      150 ... 86277500 86277550 86277600] 1725552


In [12]:
# überprüfen ob der richtige Index für den entsprechende Timer Wert gefunden wurde
# bei timer sollte 50 stehen (50ms == 1/20sec)
df_no_audio.iloc[1]

accelerometer_x   -7.167969e+00
accelerometer_y    6.801758e+00
accelerometer_z    7.861328e-01
magnetometer_x    -5.393750e+07
magnetometer_y     4.062500e+06
magnetometer_z    -5.937500e+06
count              1.000000e+00
Name: 1, dtype: float64

In [ ]:
###################### nach MS ################################################
# erstellen eines leeren dictionaries
smt_dict = {}

for variable in unpickled_df.keys():
    if "accelerometer" in variable:
        # Array wird anhand der Indizes der MS gesplittet, mit NaN values auf die selbe lenge gebracht und gestacked
        ac_split_arr = np.split(unpickled_df[variable], val_range)
        pad = len(max(ac_split_arr, key=len))
        pad_ac_split_arr = np.array([np.append(i, [[np.nan]*(pad-len(i))]) for i in ac_split_arr])
        stack_ac_split_arr = np.vstack(pad_ac_split_arr)
        
        # Zusammenfassen und in dictionary schreiben, NaN values werden ignoriert
        # maximum
        smt_dict[variable+'_max'] = np.nanmax(stack_ac_split_arr, axis = 1)
        # 95% quantile
        smt_dict[variable+'_95q'] = np.nanquantile(stack_ac_split_arr, 0.95, axis = 1)
        # minimum
        smt_dict[variable+'_min'] = np.nanmin(stack_ac_split_arr, axis = 1)
        # 5% quantile
        smt_dict[variable+'_05q'] = np.nanquantile(stack_ac_split_arr, 0.05, axis = 1)
        
    elif "magnetometer" in variable:
        # Array wird anhand der Indizes der MS gesplittet, mit NaN values auf die selbe lenge gebracht und gestacked
        ma_split_arr = np.split(unpickled_df[variable], val_range)
        pad = len(max(ma_split_arr, key=len))
        pad_ma_split_arr = np.array([np.append(i, [[np.nan]*(pad-len(i))]) for i in ma_split_arr])
        stack_ma_split_arr = np.vstack(pad_ma_split_arr)
        
        # Zusammenfassen und in dictionary schreiben, NaN values werden ignoriert
        # mean
        smt_dict[variable+'_mean'] = np.nanmean(stack_ma_split_arr, axis = 1)
        # median
        smt_dict[variable+'_med'] = np.nanmedian(stack_ma_split_arr, axis = 1)
        
    elif "audio" in variable:
        au_split_arr = np.split(unpickled_df[variable], range(audio_step, unpickled_df['audio'].size, audio_step))
        pad = len(max(au_split_arr, key=len))
        pad_au_split_arr = np.array([np.append(i, [[np.nan]*(pad-len(i))]) for i in au_split_arr])
        stack_au_split_arr = np.vstack(pad_au_split_arr)
        
        # Zusammenfassen und in DataFrame schreiben, NaN values werden ignoriert
        # maximum
        smt_dict[variable+'_max'] = np.amax(au_split_arr, axis = 1)
        # 95% quantile
        smt_dict[variable+'_95q'] = np.quantile(au_split_arr, 0.95, axis = 1)

        
print('sensor ', smt_dict['accelerometer_x_max'].size)
print('audio ', smt_dict['audio_max'].size)